In [40]:
import pandas as pd
import datetime
import os

In [41]:
# import "calendar_statement_" + year + month + day + ".csv" with year month and day being the date of last sunday
# find the day  of the last sunday
today = datetime.date.today()
sunday = today - datetime.timedelta(days=today.weekday() + 1)
year = sunday.year
month = sunday.month
day = sunday.day
# import the file
# filename = "C:/Users/WilliamFetzner/AppData/Roaming/MetaQuotes/Terminal/0C2BFA140CA8FBEFEDCADDDEDD61AA24/MQL4/Files/calendar_statement_" + str(year) + '_' + str(month) + '_' +  str(day) + ".csv"
filename_path = "C:/Users/WilliamFetzner/AppData/Roaming/MetaQuotes/Terminal/0C2BFA140CA8FBEFEDCADDDEDD61AA24/tester/files/calendar_statement_"
# filename_path

In [42]:
df = pd.read_csv("C:/Users/WilliamFetzner/Downloads/calendar-event-list.csv")
df

,Id,Start,Name,Impact,Currency
0,d1f248e0-cb0a-400b-9e82-e72dd62294ca,01/03/2017 15:00:00,ISM Manufacturing PMI,HIGH,USD
1,8823790e-8ef9-4b04-9e89-5a9eb3f488a8,01/03/2017 15:00:00,ISM Manufacturing Prices Paid,HIGH,USD
2,994b663d-8c40-4140-9897-f8532f0ec70b,01/04/2017 10:00:00,Core Harmonized Index of Consumer Prices (YoY),HIGH,EUR
3,99e70d62-059f-4b6c-837c-547913151dcb,01/04/2017 10:00:00,Harmonized Index of Consumer Prices (YoY),HIGH,EUR
4,b34d64c2-79fb-4e1b-bfaa-aa5f03b65560,01/04/2017 19:00:00,FOMC Minutes,HIGH,USD
...,...,...,...,...,...
1786,41621ff7-cc7c-4191-85c1-48aa27dddd7a,12/16/2021 12:45:00,ECB Rate On Deposit Facility,HIGH,EUR
1787,cf3b3f68-c3e3-4bc2-af0a-4418c50c8dca,12/16/2021 13:30:00,ECB Press Conference,HIGH,EUR
1788,8a793dba-482f-49ab-8e03-1edcbb6b120e,12/22/2021 13:30:00,Gross Domestic Product Annualized,HIGH,USD
1789,e1deb810-a292-4216-9f5b-7f6ffd6c5aa3,12/23/2021 13:30:00,Durable Goods Orders,HIGH,USD


In [43]:
# convert "Start" to datetime
df['datetime'] = pd.to_datetime(df['Start'])
# drop the Id column
df_dropped = df.drop(columns=['Id', 'Start'])
df_dropped['datetime'] = df_dropped['datetime'] + pd.Timedelta(hours=7)
# create a new column for the week number of the year
df_dropped['Week'] = df_dropped['datetime'].dt.isocalendar().week
# create a new column for year_week like 2024_01
df_dropped['Year_Week'] = df_dropped['datetime'].dt.strftime('%Y_%W')
# sort the dataframe by datetime
df_dropped = df_dropped.sort_values(by='datetime', ascending=True)
# change the column names to match news_df
df_dropped = df_dropped.rename(columns={'Name': 'Event'})
df_dropped

,Event,Impact,Currency,datetime,Week,Year_Week
0,ISM Manufacturing PMI,HIGH,USD,2017-01-03 22:00:00,1,2017_01
1,ISM Manufacturing Prices Paid,HIGH,USD,2017-01-03 22:00:00,1,2017_01
2,Core Harmonized Index of Consumer Prices (YoY),HIGH,EUR,2017-01-04 17:00:00,1,2017_01
3,Harmonized Index of Consumer Prices (YoY),HIGH,EUR,2017-01-04 17:00:00,1,2017_01
4,FOMC Minutes,HIGH,USD,2017-01-05 02:00:00,1,2017_01
...,...,...,...,...,...,...
1786,ECB Rate On Deposit Facility,HIGH,EUR,2021-12-16 19:45:00,50,2021_50
1787,ECB Press Conference,HIGH,EUR,2021-12-16 20:30:00,50,2021_50
1788,Gross Domestic Product Annualized,HIGH,USD,2021-12-22 20:30:00,51,2021_51
1789,Durable Goods Orders,HIGH,USD,2021-12-23 20:30:00,51,2021_51


In [44]:
path_to_news = 'C:/Users/WilliamFetzner/Documents/Trading/calendar_df_full.csv'
news_df = pd.read_csv(path_to_news)
if 'Unnamed: 0' in news_df.columns:
    news_df.drop('Unnamed: 0', axis=1, inplace=True)
if 'Date' in news_df.columns:
    news_df.drop('Date', axis=1, inplace=True)
news_df.loc[:, 'datetime'] = pd.to_datetime(news_df['datetime'])
cols = news_df.columns
news_df

,Event,Impact,Currency,Week,Year_Week,datetime
0,(Spain) Markit Manufacturing PMI,High,EUR,1,2022_01,2022-01-03 17:15:00
1,(Austria) Unemployment Rate,High,EUR,1,2022_01,2022-01-03 17:30:00
2,(Italy) IHS Markit Manufacturing PMI,High,EUR,1,2022_01,2022-01-03 17:45:00
3,(Germany) Markit Manufacturing PMI,High,EUR,1,2022_01,2022-01-03 17:55:00
4,(Greece) Markit Manufacturing PMI,High,EUR,1,2022_01,2022-01-03 18:00:00
...,...,...,...,...,...,...
3194,(United States) Personal Spending MoM,High,USD,13,2024_13,2024-03-29 15:30:00
3195,(United States) Personal Income MoM,High,USD,13,2024_13,2024-03-29 15:30:00
3196,(United States) Core PCE Price Index YoY,High,USD,13,2024_13,2024-03-29 15:30:00
3197,(United States) Core PCE Price Index MoM,High,USD,13,2024_13,2024-03-29 15:30:00


In [52]:
# reorder df columns to match news_df columns
df_dropped = df_dropped[cols]
# concat news_df and df_dropped
all_news_df = pd.concat([news_df, df_dropped], ignore_index=True)
# drop any duplicates
all_news_df = all_news_df.drop_duplicates()
all_news_df = all_news_df.sort_values(by='datetime', ascending=True)
all_news_df

,Event,Impact,Currency,Week,Year_Week,datetime
3200,ISM Manufacturing Prices Paid,HIGH,USD,1,2017_01,2017-01-03 22:00:00
3199,ISM Manufacturing PMI,HIGH,USD,1,2017_01,2017-01-03 22:00:00
3201,Core Harmonized Index of Consumer Prices (YoY),HIGH,EUR,1,2017_01,2017-01-04 17:00:00
3202,Harmonized Index of Consumer Prices (YoY),HIGH,EUR,1,2017_01,2017-01-04 17:00:00
3203,FOMC Minutes,HIGH,USD,1,2017_01,2017-01-05 02:00:00
...,...,...,...,...,...,...
3194,(United States) Personal Spending MoM,High,USD,13,2024_13,2024-03-29 15:30:00
3195,(United States) Personal Income MoM,High,USD,13,2024_13,2024-03-29 15:30:00
3196,(United States) Core PCE Price Index YoY,High,USD,13,2024_13,2024-03-29 15:30:00
3197,(United States) Core PCE Price Index MoM,High,USD,13,2024_13,2024-03-29 15:30:00


In [46]:
for i in all_news_df.Year_Week.unique():
    df_wk = all_news_df[all_news_df['Year_Week'] == i]
    # sort the dataframe by datetime
    df_wk = df_wk.sort_values(by='datetime', ascending=True)
    # grab the datetime of the first day of the week
    first_day = df_wk['datetime'].iloc[0]
    # find out what the previous sunday was for first_day
    sunday = first_day - datetime.timedelta(days=first_day.weekday() + 1)
    year = sunday.year
    month = sunday.month
    day = sunday.day
    # get the filename ready
    filename = f"{filename_path}{year}_{month}_{day}.csv"
    # print(filename)
    # if the file doesn't already exist then save it
    if not os.path.exists(filename):
        df_wk.to_csv(filename, index=False)

In [47]:
file_location = 'C:/Users/WilliamFetzner/AppData/Roaming/MetaQuotes/Terminal/0C2BFA140CA8FBEFEDCADDDEDD61AA24/MQL4/Files/'
files = [f for f in os.listdir(file_location) if '20' in f]

In [53]:
# Find the Monday of the current week
today = datetime.datetime.today()

# Get the maximum date from all_news_df
max_date = all_news_df['datetime'].max()

# Calculate the Monday of the week of the max_date
max_date_monday = max_date - datetime.timedelta(days=max_date.weekday())

# Calculate the Monday of the current week
current_week_monday = today - datetime.timedelta(days=today.weekday())
next_week_monday = current_week_monday + datetime.timedelta(days=7)

# Check for any new weeks that need to be added to the full calendar df
for monday in pd.date_range(start=max_date_monday, end=next_week_monday, freq='W-MON'):
    sunday = monday - datetime.timedelta(days=1)
    sunday_str = f"{sunday.year}_{sunday.month}_{sunday.day}"
    print(sunday_str)
    # Read in the new event csv
    for file in files:
        if sunday_str in file:
            calendar_df = pd.read_csv(file_location + file)
            calendar_df['datetime'] = pd.to_datetime(calendar_df['Date'], format='%Y.%m.%d %H:%M:%S')
            calendar_df = calendar_df.drop(columns=['Date'])
            # create a new column for the week number of the year
            calendar_df['Week'] = calendar_df['datetime'].dt.isocalendar().week
            # create a new column for year_week like 2024_01
            calendar_df['Year_Week'] = calendar_df['datetime'].dt.strftime('%Y_%W')
            all_news_df = pd.concat([all_news_df, calendar_df], ignore_index=True)
            # Sort by datetime
            all_news_df = all_news_df.sort_values(by=['datetime'])
            print(f"New events added for {sunday_str}")
            # Save the all_news_df to a csv
            # all_news_df.to_csv('all_news_df.csv', index=False)
            break
    else:
        # Error message - the latest calendar file was not found
        print(f"Error: {sunday_str} not found in files")
all_news_df

2024_3_24
New events added for 2024_3_24
2024_3_31
New events added for 2024_3_31
2024_4_7
New events added for 2024_4_7


,Event,Impact,Currency,Week,Year_Week,datetime
0,ISM Manufacturing Prices Paid,HIGH,USD,1,2017_01,2017-01-03 22:00:00
1,ISM Manufacturing PMI,HIGH,USD,1,2017_01,2017-01-03 22:00:00
2,Core Harmonized Index of Consumer Prices (YoY),HIGH,EUR,1,2017_01,2017-01-04 17:00:00
3,Harmonized Index of Consumer Prices (YoY),HIGH,EUR,1,2017_01,2017-01-04 17:00:00
4,FOMC Minutes,HIGH,USD,1,2017_01,2017-01-05 02:00:00
...,...,...,...,...,...,...
5079,(United States) PPI MoM,High,USD,15,2024_15,2024-04-11 15:30:00
5080,(Euro Area) ECB Press Conference,High,EUR,15,2024_15,2024-04-11 15:45:00
5081,(Germany) CPI,High,EUR,15,2024_15,2024-04-12 09:00:00
5082,(Spain) CPI,High,EUR,15,2024_15,2024-04-12 10:00:00


In [54]:
# save all_news_df back to csv 
all_news_df.to_csv(path_to_news, index=False)